In [1]:
from collections import Counter
from itertools import chain, combinations
from tqdm import tqdm
import numpy as np

In [27]:
class Machine():

    def __init__(self, line: list[str]):
        self.indicators = line[0].replace("[", "").replace("]", "")
        self.indicators_num = [i for i, ch in enumerate(self.indicators) if ch == "#"]

        buttons = [item.replace("(", "").replace(")", "") for item in line[1:-1]]
        buttons = [[int(x) for x in item.split(",")] for item in buttons]
        self.buttons = buttons

        self.joltage_reqs = np.array([int(x) for x in line[-1].replace("{", "").replace("}", "").split(",")])

        button_vectors = []
        for id in range(len(buttons)):
            button_vectors.append(np.array([1 if j in buttons[id] else 0 for j in range(len(self.joltage_reqs))]))
        self.button_vectors = button_vectors

        self.button_array = np.array(self.button_vectors).T

    def __repr__(self):
        return f"Machine(indicators={self.indicators},\n indicators_num={self.indicators_num},\n buttons={self.buttons},\n button_vectors={self.button_vectors},\n joltage_reqs={self.joltage_reqs})"

In [3]:
def read_data_10(file_name):
    with open(file_name) as my_file:
        input = my_file.read()
    lines = input.splitlines()
    lines = [line.split(" ") for line in lines if line.strip()]

    machines = []
    for line in lines:
        machine = Machine(line)
        machines.append(machine)
    machines

    return machines

In [28]:
machines_test = read_data_10("../data/input10_test.txt")
machines_puzzle = read_data_10("../data/input10_puzzle.txt")
machines = machines_test
machines

[Machine(indicators=.##.,
  indicators_num=[1, 2],
  buttons=[[3], [1, 3], [2], [2, 3], [0, 2], [0, 1]],
  button_vectors=[array([0, 0, 0, 1]), array([0, 1, 0, 1]), array([0, 0, 1, 0]), array([0, 0, 1, 1]), array([1, 0, 1, 0]), array([1, 1, 0, 0])],
  joltage_reqs=[3 5 4 7]),
 Machine(indicators=...#.,
  indicators_num=[3],
  buttons=[[0, 2, 3, 4], [2, 3], [0, 4], [0, 1, 2], [1, 2, 3, 4]],
  button_vectors=[array([1, 0, 1, 1, 1]), array([0, 0, 1, 1, 0]), array([1, 0, 0, 0, 1]), array([1, 1, 1, 0, 0]), array([0, 1, 1, 1, 1])],
  joltage_reqs=[ 7  5 12  7  2]),
 Machine(indicators=.###.#,
  indicators_num=[1, 2, 3, 5],
  buttons=[[0, 1, 2, 3, 4], [0, 3, 4], [0, 1, 2, 4, 5], [1, 2]],
  button_vectors=[array([1, 1, 1, 1, 1, 0]), array([1, 0, 0, 1, 1, 0]), array([1, 1, 1, 0, 1, 1]), array([0, 1, 1, 0, 0, 0])],
  joltage_reqs=[10 11 11  5 10  5])]

## Part 1

In [363]:
def button_presses_result(buttons: list[ list[int]]) -> list:
    button_sum_raw = list(chain(*buttons))
    button_sum = []
    for key, value in Counter(button_sum_raw).items():
        if value % 2 != 0:
            button_sum.append(key)

    return sorted(button_sum)

assert button_presses_result([[3], [1, 3], [2]]) == [1, 2]
assert button_presses_result([[1, 3], [2, 3], [0, 1], [0,1]]) == [1, 2]
assert button_presses_result([[3], [2], [2, 3], [0, 2], [0, 1]]) == [1, 2]
assert button_presses_result([[1, 3], [2, 3]]) == [1, 2]
assert button_presses_result([[0, 4], [0, 1, 2], [1, 2, 3, 4]]) == [3]
assert button_presses_result([[0, 3, 4], [0, 1, 2, 4, 5],]) == [1, 2, 3, 5]

In [364]:
def minimal_button_presses(machine: Machine) -> int:
    button_presses = 0
    if machine.indicators_num in machine.buttons:
        button_presses = 1

    subset_buttons = 2
    while button_presses == 0:
        button_combinations = combinations(machine.buttons, r=subset_buttons)
        for button_combination in button_combinations:
            if button_presses_result(button_combination) == machine.indicators_num:
                button_presses = subset_buttons
        subset_buttons += 1

    return button_presses

In [365]:
total_button_presses = 0
for machine in tqdm(machines_test):
    total_button_presses += minimal_button_presses(machine)
total_button_presses

100%|██████████| 3/3 [00:00<00:00, 1715.70it/s]


7

In [366]:
total_button_presses = 0
for machine in tqdm(machines_puzzle):
    total_button_presses += minimal_button_presses(machine)
total_button_presses

100%|██████████| 166/166 [00:00<00:00, 1836.21it/s]


486

## Part 2

### Test

In [34]:
id = 0
machine = machines_test[id]
solutions = [np.array([1, 3, 0, 3, 1, 2]),
             np.array([2, 5, 0, 5, 0]),
             np.array([5, 0, 5, 1])]
for id, machine in enumerate(machines_test):
    assert all(machine.button_array @ solutions[id] == machine.joltage_reqs)

[np.sum(array) for array in solutions]

[np.int64(10), np.int64(12), np.int64(11)]

In [207]:
def find_biggest_multiplier_for_vector(joltage_reqs, button_vector):
    multiplier = 0
    while all((joltage_reqs - button_vector*multiplier) > 0):
        multiplier += 1
    return multiplier

def max_multipliers_for_machine(machine):
    max_multipliers = []
    for button_vector in machine.button_vectors:
        max_multipliers.append(find_biggest_multiplier_for_vector(machine.joltage_reqs, button_vector))
    return max_multipliers

In [208]:
max_multipliers_for_machine(machines_test[0])

[7, 5, 4, 4, 3, 3]

In [279]:
from itertools import product

def find_solutions_through_combos(machine):
    max_multipliers = max_multipliers_for_machine(machine)
    # print(max_multipliers)

    test_list = [range(value+1) for value in max_multipliers]
    output_generator = product(*test_list)

    solution_list = []

    for item in output_generator:
        if all(machine.button_array @ np.array(item) == machine.joltage_reqs):
            solution_list.append(item)

    return solution_list


In [318]:
button_presses = []
for machine in tqdm(machines_test):
    solutions = find_solutions_through_combos(machine)
    print(solutions)
    button_presses.append(min([sum(solution) for solution in solutions]))

button_presses

100%|██████████| 3/3 [00:00<00:00, 52.63it/s]

[(1, 2, 0, 4, 0, 3), (1, 3, 0, 3, 1, 2), (1, 4, 0, 2, 2, 1), (1, 5, 0, 1, 3, 0), (2, 2, 1, 3, 0, 3), (2, 3, 1, 2, 1, 2), (2, 4, 1, 1, 2, 1), (2, 5, 1, 0, 3, 0), (3, 2, 2, 2, 0, 3), (3, 3, 2, 1, 1, 2), (3, 4, 2, 0, 2, 1), (4, 2, 3, 1, 0, 3), (4, 3, 3, 0, 1, 2), (5, 2, 4, 0, 0, 3)]
[(0, 7, 2, 5, 0), (1, 6, 1, 5, 0), (2, 5, 0, 5, 0)]
[(0, 5, 5, 6), (1, 4, 5, 5), (2, 3, 5, 4), (3, 2, 5, 3), (4, 1, 5, 2), (5, 0, 5, 1)]


[10, 12, 11]

In [321]:
def look_for_solutions(joltage_reqs, button_vectors, solutions, multiplier_vector = []):
    button_vector = button_vectors[0]
    # print(button_vector)

    multiplier = 0
    while True:
        new_joltage_req = joltage_reqs - button_vector * multiplier
        new_multiplier_vector = multiplier_vector + [multiplier]
        if all(new_joltage_req == 0):
            solutions.append(new_multiplier_vector)
            # print(f"found_solution: {new_multiplier_vector}")
        elif any(new_joltage_req < 0):
            break
        else:
            if len(button_vectors) > 1:
                solutions = look_for_solutions(new_joltage_req, button_vectors[1:], solutions, new_multiplier_vector)
        multiplier += 1
    
    return solutions


In [322]:
def find_solutions(machine):
    solutions = look_for_solutions(machine.joltage_reqs, machine.button_vectors, [])

    for id, solution in enumerate(solutions):
        if len(solution) != len(machine.button_vectors):
            solutions[id] = solution + [0] * (len(machine.button_vectors) - len(solution))

    return solutions

In [324]:
button_presses = []
for machine in tqdm(machines_test):
    solutions = find_solutions(machine)
    print(solutions)
    button_presses.append(min([sum(solution) for solution in solutions]))

print(button_presses)
print(sum(button_presses))

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 75.42it/s]

[[1, 2, 0, 4, 0, 3], [1, 3, 0, 3, 1, 2], [1, 4, 0, 2, 2, 1], [1, 5, 0, 1, 3, 0], [2, 2, 1, 3, 0, 3], [2, 3, 1, 2, 1, 2], [2, 4, 1, 1, 2, 1], [2, 5, 1, 0, 3, 0], [3, 2, 2, 2, 0, 3], [3, 3, 2, 1, 1, 2], [3, 4, 2, 0, 2, 1], [4, 2, 3, 1, 0, 3], [4, 3, 3, 0, 1, 2], [5, 2, 4, 0, 0, 3]]
[[0, 7, 2, 5, 0], [1, 6, 1, 5, 0], [2, 5, 0, 5, 0]]
[[0, 5, 5, 6], [1, 4, 5, 5], [2, 3, 5, 4], [3, 2, 5, 3], [4, 1, 5, 2], [5, 0, 5, 1]]
[10, 12, 11]
33


### Sympy

In [357]:
from sympy.solvers.diophantine.diophantine import diop_linear
from sympy import symbols
from sympy import simplify
x, y, z, t_0, t= symbols("x, y, z, t_0, t", integer=True)
solution = diop_linear(2*x + 3*y - 5, param=t)
solution


(3*t_0 - 5, 5 - 2*t_0)

In [362]:
simplify(solution.subs({t_0: 9}))

(22, -13)

### Puzzle

In [329]:
solutions = find_solutions(machines_puzzle[3])
solutions

[[6, 9, 0, 5],
 [7, 9, 1, 4],
 [8, 9, 2, 3],
 [9, 9, 3, 2],
 [10, 9, 4, 1],
 [11, 9, 5, 0]]

In [296]:
button_presses = []
for machine in tqdm(machines_puzzle):
    solutions = find_solutions(machine)
    print(solutions)
    button_presses.append(min([sum(solution) for solution in solutions]))

print(button_presses)
print(sum(button_presses))

  0%|          | 0/166 [00:17<?, ?it/s]


KeyboardInterrupt: 

In [41]:
joltage_reqs = machine.joltage_reqs
# while all(joltage_reqs > 0)
for button_id, button_vector in enumerate(machine.button_vectors):
    button_presses = [0]*len(machine.button_vectors)
    new_joltage_reqs = joltage_reqs - button_vector
    button_presses[button_id] += 1
    print(new_joltage_reqs, button_presses)

[ 9 10 10  4  9  5] [1, 0, 0, 0]
[ 9 11 11  4  9  5] [0, 1, 0, 0]
[ 9 10 10  5  9  4] [0, 0, 1, 0]
[10 10 10  5 10  5] [0, 0, 0, 1]
